## SRJ

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore")
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        #print('HF stat:', hf_state)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('Wires1 Invert removed are ', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)) )
                #print('Wires2 Invert removed are',list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    dev2 = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev2)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        counts = measure(states[-1])
        print('The latest added state counts', counts)
        print('No of states are', len(counts))
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=20) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668944
Total excitations are 26
The adapt iteration now is 0
Highest gradient excitation is [2, 3, 6, 7]
Final updated parameters: [1.12299241]
Final cost: -1.430083670840328
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(1.12)─┤  State
3: ──X─├FermionicDoubleExcitation(1.12)─┤  State
6: ────├FermionicDoubleExcitation(1.12)─┤  State
7: ────╰FermionicDoubleExcitation(1.12)─┤  State
The latest added state counts {np.str_('11000011'): np.int64(297), np.str_('11110000'): np.int64(703)}
No of states are 2
The adapt iteration now is 1
Highest gradient excitation is [0, 3, 5, 6]
Final updated parameters: [1.09716154 1.37775771]
Final cost: -1.5694603835520364
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(1.38)─┤  State
1: ──X──────────────────────────────────├FermionicDoubleExcitation(1.

# New idea

In [15]:
energies[8:13]

[-1.8843560329960767,
 -1.8844361654274617,
 -1.8844361671882441,
 -1.8844364522255297,
 -1.8844364556799282]

In [7]:
ash_excitation[8:13]

[[0, 4], [0, 1, 4, 5], [0, 1, 6, 7], [2, 3, 4, 5], [0, 3, 4, 7]]

In [13]:
ash_excitation[:9]

[[2, 3, 6, 7],
 [0, 3, 5, 6],
 [0, 1, 4, 5],
 [1, 2, 4, 7],
 [1, 5],
 [2, 6],
 [3, 7],
 [2, 3, 6, 7],
 [0, 4]]

In [12]:
ash_excitation

[[2, 3, 6, 7],
 [0, 3, 5, 6],
 [0, 1, 4, 5],
 [1, 2, 4, 7],
 [1, 5],
 [2, 6],
 [3, 7],
 [2, 3, 6, 7],
 [0, 4],
 [0, 1, 4, 5],
 [0, 1, 6, 7],
 [2, 3, 4, 5],
 [0, 3, 4, 7],
 [1, 2, 5, 6],
 [1, 3, 5, 7],
 [0, 2, 4, 6],
 [0, 3, 5, 6],
 [2, 6],
 [1, 3, 5, 7],
 [0, 4]]

## UCCSD - VQE

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore")
from scipy.optimize import minimize
optimizer = qml.GradientDescentOptimizer(stepsize=1.0)
 
ash_excitation = []
energies = []
excitations= []
old_grad = []
 
X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity
 
 
 
 
 
 
def uccsd_sri(symbols, coordinates, active_electrons, active_orbitals, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
 
    hf_state = qchem.hf_state(active_electrons, qubits)  # HF state - Our starting state
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H):
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
 
 
 
    
 
    
    dev2 = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev2)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
 
    
 
    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    #print('Singles are', singles)
    #op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    #op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (singles) + (doubles)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    ash_excitation = operator_pool.copy()  #Ash excitation is the operator pool
    params = np.zeros(len(ash_excitation), requires_grad=True)
 
    #print('The initial params are', len(params))
    print('Excitations are', ash_excitation)
 
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def UCCSD_an(hf_state, ash_excitation, params):
        qml.BasisState(hf_state, wires=range(qubits))
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('The wires1', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
                #print('The wires2', list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                #qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.DoubleExcitation(params[i], wires=ash_excitation[i])
            elif len(ash_excitation[i]) == 2:
                #print('Single excitation', ash_excitation[i])
                #print('The first part of wires', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
                #qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
                qml.SingleExcitation(params[i], wires=ash_excitation[i])
        return qml.expval(H)
    
    def cost(params):
        energy = UCCSD_an(hf_state, ash_excitation, params)
        return energy
 
    def callback(params):
        #print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    
    #for n in range(500):
    #    params, energy = optimizer.step_and_cost(cost, params)
    #    if n % 2 == 0:
    #        print("step = {:},  E = {:.8f} Ha".format(n, energy))
    #print('The final params are', params)
    #print('The final energy is', energy)
    result = minimize(cost, params, method='BFGS', tol = 1e-12, callback = callback, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-8})
    print("Final updated parameters:", result.x)
    print("Final cost:", result.fun)
    
    energies = result.fun
    #energies = energy
    
        
    return energies, params
 
 
 
 
symbols  = [ 'H', 'H', 'H', 'H']   #Symbols - Molecule here
print('H4-3A-GS-BFGS-sto6g')      
r_bohr = 1.8897259886              #Since Pennylane takes input in Bohr, you need the multiplying factor
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])  # Coordinates of your molecule
 
 
 
electrons = 4 # 4 electrons for H4
orbitals = 8  # Spin orbitals i.e. qubits
charge = 0    # Neutral molecule
 
active_electrons = 4  #Since we are working on small electron - consider everything to be active
active_orbitals = 4 #Thinkng it is spatial
 
 
energies, params = uccsd_sri(symbols, coordinates, active_electrons, active_orbitals, shots = None)

H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668933
Total excitations are 26
Excitations are [[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7], [0, 1, 4, 5], [0, 1, 4, 7], [0, 1, 5, 6], [0, 1, 6, 7], [0, 2, 4, 6], [0, 3, 4, 5], [0, 3, 4, 7], [0, 3, 5, 6], [0, 3, 6, 7], [1, 2, 4, 5], [1, 2, 4, 7], [1, 2, 5, 6], [1, 2, 6, 7], [1, 3, 5, 7], [2, 3, 4, 5], [2, 3, 4, 7], [2, 3, 5, 6], [2, 3, 6, 7]]
Current cost: -1.873924999363083

Current cost: -1.87872608755612

Current cost: -1.881563350647929

Current cost: -1.8817152956286662

Current cost: -1.8818434985027501

Current cost: -1.8818546578907913

Current cost: -1.8818673442852432

Current cost: -1.8818683067515243

Current cost: -1.8818694413257182

Current cost: -1.8818709363353676

Current cost: -1.881873392996667

Current cost: -1.8818777149169286

Current cost: -1.8818857168505647

Current cost: -1.8819009565363358

Current cost: -1.8819292538808747

Current cost

## L-BFGS optimizer

In [ ]:
p_mycode = -1.8843126366466898  #qml.Single/Double
p_ocode = -1.88491980           #qml.Original code
p_ocode_F = - 1.8847077939970456 #qml. FSE/FDE
q_code = -1.8842278703722253 -1.884221040078635

## Powell optimizer

In [ ]:
p_mycode =    #qml.Single/Double
p_ocode = -1.88491980           #qml.Original code
p_ocode_F =  #qml. FSE/FDE
q_code = -1.8842278703722253 -1.884221040078635

# BFGS

p_ocode = -1.884312636649352
p_ocode_F = -1.8842441915391785
q_code = -1.8843320781188295